In [1]:
%reset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


c:\users\pradhyum\appdata\local\programs\python\python37\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
from __future__ import print_function
import tensorflow 
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Activation, TimeDistributed
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from urllib.request import urlopen
import urllib.request
import re 

In [3]:
def get_lyrics(artist,song_title):
    artist = artist.lower()
    song_title = song_title.lower()
    # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').replace('<br/>','').strip()
        return lyrics
    except Exception as e:
        return "Exception occurred \n" +str(e)

In [4]:

def create_corpus(artist_name):
    url = 'http://lyrics.wikia.com/wiki/' + artist_name
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    count=0
    data=[]

    # Parse the data to get list of songs and urls
    for album in soup.find_all(class_='album-art'):
        count += 1
        for song in album.find_next('ol').children:
            try:
                a = re.search('\:(.*)', song.b.a['href'])
                data.append({
                    'url': song.b.a['href'],
                    'name': a.group(1)
                })
            except:
                continue
    df = pd.DataFrame(data, columns=["url","name"])
    print('Collecting {0} songs for {1}'.format(df.shape[0],artist_name))
    corpus = ''
    for ind, row in df.iterrows():
        #print('Collecting song: {}'.format(row['name']))
        corpus += '\n' + get_lyrics(artist_name,row['name'])
    print('Writing song corpus for {}'.format(artist_name))
    with open(artist_name+"_corpus.txt", "w") as text_file:
        text_file.write(corpus)



In [5]:
artist_name = 'GOT'

In [6]:
def get_corpus_sequence(artist_name, level='word'):
    from pathlib import Path
    my_file = Path(artist_name+'_corpus.txt')
    if not my_file.is_file():
        print('Corpus doesn\'t already exit for {}'.format(artist_name))
        create_corpus(artist_name)
    with open(artist_name+'_corpus.txt','r',encoding='utf8') as f:
        print('Gathering and processing sequence or text')
        try:
            text = f.read().strip().lower().replace('\n',' \n ')
        except:
            text = f.read().encode('utf-8').strip().lower().replace('\n',' \n ')
    if level =='word':
        text_seq = [word for word in text.split(' ') if (word.strip != '' and word != '')]
    elif level=='char':
        text_seq = [char for char in text]
    return text_seq

In [7]:
text_seq = get_corpus_sequence(artist_name,level='word')

Gathering and processing sequence or text


In [8]:
text_seq

['[first',
 'scene',
 'opens',
 'with',
 'three',
 'rangers',
 'riding',
 'through',
 'a',
 'tunnel,',
 'leaving',
 'the',
 'wall,',
 'and',
 'going',
 'into',
 'the',
 'woods.',
 '(eerie',
 'music',
 'in',
 'background)',
 'one',
 'ranger',
 'splits',
 'off',
 'and',
 'finds',
 'a',
 'campsite',
 'full',
 'of',
 'mutilated',
 'bodies,',
 'including',
 'a',
 'child',
 'hanging',
 'from',
 'a',
 'tree',
 'branch.',
 'a',
 'birds-eye',
 'view',
 'shows',
 'the',
 'bodies',
 'arranged',
 'in',
 'a',
 'shield-like',
 'pattern.',
 'the',
 'ranger',
 'rides',
 'back',
 'to',
 'the',
 'other',
 'two.]',
 '\n',
 'waymar',
 'royce:',
 'what',
 'd’you',
 'expect?',
 'they’re',
 'savages.',
 'one',
 'lot',
 'steals',
 'a',
 'goat',
 'from',
 'another',
 'lot',
 'and',
 'before',
 'you',
 'know',
 'it,',
 'they’re',
 'ripping',
 'each',
 'other',
 'to',
 'pieces.',
 '\n',
 'will:',
 'i’ve',
 'never',
 'seen',
 'wildlings',
 'do',
 'a',
 'thing',
 'like',
 'this.',
 'i’ve',
 'never',
 'seen',
 'a',

In [9]:
# Calculate word frequency
word_freq = {}
for word in text_seq:
    word_freq[word] = word_freq.get(word, 0) + 1
MIN_WORD_FREQUENCY = 4
ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_seq)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

text_seq = [word for word in text_seq if word not in ignored_words]
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))


Unique words before ignoring: 25979
Ignoring words with frequency < 4
Unique words after ignoring: 8073


In [10]:
# cut the text in semi-redundant sequences of SEQUENCE_LEN words
INPUT_SEQUENCE_LEN = 5
OUTPUT_SEQUENCE_LEN = 1
STEP = OUTPUT_SEQUENCE_LEN
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_seq) - INPUT_SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_seq[i: i+INPUT_SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_seq[i: i + INPUT_SEQUENCE_LEN])
        next_words.append(text_seq[i + INPUT_SEQUENCE_LEN:i + INPUT_SEQUENCE_LEN + OUTPUT_SEQUENCE_LEN ])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))
for i in range(10):
    rand_index = np.random.randint(0,len(sentences))
    
    print('Randome sentence: {}'.format(' '.join(sentences[rand_index])))
    print('Corresponding next words: {}'.format(next_words[rand_index]))
sentences = sentences[-5000:]
next_words = next_words[-5000:]

Ignored sequences: 0
Remaining sequences: 455449
Randome sentence: but if the entire north
Corresponding next words: ['has']
Randome sentence: 
 ser barristan: a royal
Corresponding next words: ['pardon']
Randome sentence: and approaches yara. 
 daenerys:
Corresponding next words: ['you']
Randome sentence: the girl she wore to
Corresponding next words: ['the']
Randome sentence: for 300 years. there's not
Corresponding next words: ['a']
Randome sentence: thing that might turn it
Corresponding next words: ['are']
Randome sentence: wounds that needed 
 cersei:
Corresponding next words: ['what']
Randome sentence: my son? 
 grand maester
Corresponding next words: ['pycelle:']
Randome sentence: searches down gendry's body, and
Corresponding next words: ['he']
Randome sentence: and margaery. 
 olenna: so
Corresponding next words: ['their']


In [11]:
len(sentences)

5000

In [12]:
class seqSentenceGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, sentences=sentences, next_words=next_words,word_indices=word_indices, indices_word=indices_word, batch_size=64, shuffle=True):
        'Initialization'
        self.dim = (len(sentences),INPUT_SEQUENCE_LEN,len(words))
        self.sentences = sentences
        self.next_words = next_words
        self.word_indices = word_indices
        self.indices_word = indices_word
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.dim[0]//self.batch_size

    def __getitem__(self, index, seq2seq=False):
        'Generate one batch of data'
        # Generate data
        X, y = next(self.__data_generation(self.sentences,self.next_words))
        return X, y

    '''def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)'''
            
    def __data_generation(self, sentences, next_words):
        for i, sentence in enumerate(sentences):
            x = np.zeros((1,INPUT_SEQUENCE_LEN, len(words)), dtype=np.int8)
            y = np.zeros((1,INPUT_SEQUENCE_LEN, len(words)), dtype=np.int8)
            for tx, word in enumerate(sentence):
                x[0, tx, self.word_indices[word]] = 1
            for ty, word in enumerate(next_words[i]):
                y[0, ty, self.word_indices[word]] = 1
            yield x,y

In [13]:
class SentenceGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, sentences=sentences, next_words=next_words,word_indices=word_indices, indices_word=indices_word, batch_size=64, shuffle=True, INPUT_SEQUENCE_LEN=INPUT_SEQUENCE_LEN):
        'Initialization'
        self.dim = (len(sentences),INPUT_SEQUENCE_LEN,len(words))
        self.sentences = sentences
        self.next_words = next_words
        self.word_indices = word_indices
        self.indices_word = indices_word
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.dim[0]//self.batch_size

    def __getitem__(self, index, seq2seq=False):
        'Generate one batch of data'
        # Generate data
        X, y = next(self.__data_generation(self.sentences,self.next_words))
        return X, y
    
    def __data_generation(self, sentences, next_words):
        for i, sentence in enumerate(sentences):
            X_ngram = np.zeros((1,INPUT_SEQUENCE_LEN, len(words)), dtype=np.int8)
            y_ngram = np.zeros((1, len(words)), dtype=np.int8)
            for tx, word in enumerate(sentence):
                X_ngram[i, tx, self.word_indices[word]] = 1
            y_ngram[i, self.word_indices[word]] = 1
            yield X_ngram, y_ngram


In [14]:
X_ngram = np.zeros((len(sentences),INPUT_SEQUENCE_LEN, len(words)), dtype=np.int8)
y_ngram = np.zeros((len(sentences), len(words)), dtype=np.int8)
for i, sentence in enumerate(sentences):
    for tx, word in enumerate(sentence):
        X_ngram[i, tx, word_indices[word]] = 1
    next_word = next_words[i][0] 
    y_ngram[i, word_indices[next_word]] = 1
    
X_ngram.shape, y_ngram.shape


((5000, 5, 8073), (5000, 8073))

In [15]:
len(next_words)

5000

In [18]:
# build the model: a single LSTM
def create_model(seq2seq=True, enc_dec_attention=False):
    print('Build model...')
    if seq2seq:
        inputs = Input(shape=(INPUT_SEQUENCE_LEN,len(words)))
        encoder = LSTM(units=128, return_sequences=True, name='encoder_LSTM', dropout=0.5)(inputs)
        decoder = LSTM(units=128, return_sequences=True, name='decoder_LSTM', dropout=0.5)(encoder)
        if not enc_dec_attention:
            outputs = TimeDistributed(Dense(len(words), activation='softmax', name='Time Distributed Dense'))(decoder)
        else:
            context = tensorflow.keras.layers.Attention(name='attention')([decoder,encoder])
            concatenated = tensorflow.keras.layers.Concatenate(name='concatenate_attention')([context,decoder])
            outputs = TimeDistributed(Dense(len(words), activation='softmax'))(concatenated)

        model = Model(inputs=inputs,outputs=outputs)
        return model
    else:
        inputs = Input(shape=(INPUT_SEQUENCE_LEN,len(words)))
        lstm1 = LSTM(units=128, return_sequences=False, name='encoder_LSTM1')(inputs)
        #lstm2 = LSTM(units=128, return_sequences=False, name='encoder_LSTM2', dropout=0.4)(lstm1)
        outputs = Dense(len(words), activation='softmax')(lstm1)
        model = Model(inputs=inputs, outputs=outputs)
        return model


In [19]:
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tensorflow.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tensorflow.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tensorflow.Session(config=config_proto)
set_session(session)
model = create_model(seq2seq=False,enc_dec_attention=True)
from tensorflow.keras.optimizers import RMSprop, Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
model.summary()

Build model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 8073)]         0         
_________________________________________________________________
encoder_LSTM1 (LSTM)         (None, 128)               4199424   
_________________________________________________________________
dense (Dense)                (None, 8073)              1041417   
Total params: 5,240,841
Trainable params: 5,240,841
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Use genrator to create batches of training examples to train the model

In [21]:
def sample(preds, seq2seq=False, temperature=1.0):
    # helper function to sample an index from a probability array
    if not seq2seq:
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    else:
        samples = []
        for i in range(OUTPUT_SEQUENCE_LEN):
            preds = preds_orig[0,i,:].astype('float64')
            preds = np.log(preds) / temperature
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            samples.append(np.argmax(np.random.multinomial(1,preds,1)))
        return samples

In [22]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    if epoch%1 == 0:
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text_seq) - INPUT_SEQUENCE_LEN - 1)
        generated = ''
        sentence = ' '.join(text_seq[start_index: start_index + INPUT_SEQUENCE_LEN])
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(0,50, STEP):
            x_pred = np.zeros((1, INPUT_SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence.split(' ')):
                x_pred[0, t, word_indices[word]] = 1.
            #print('Here the sentence is {}'.format(sentence))

            preds = model.predict(x_pred)
            #next_indices = sample(preds, temperature=1.0)
            #try:
            #    next_words = ' '.join([indices_word[ind] for ind in next_indices])
            #except:
            #    next_words = indices_word[next_indices]
            next_words = indices_word[sample(preds)]
            sentence = ' '.join(sentence.split(' ')[STEP:])+ ' ' +next_words
            #print('After appending next word the sentence is: {}'.format(sentence))
            sys.stdout.write(' ' + next_words)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
weights_seq_output = np.zeros((INPUT_SEQUENCE_LEN,len(words)))
weights_seq_output[range(OUTPUT_SEQUENCE_LEN),:] = 1
model_history = model.fit_generator(generator=SentenceGenerator(),
                                    epochs=1,
                                    callbacks=[print_callback],
                                    class_weight=weights_seq_output,
                                    use_multiprocessing=False,
                                    max_queue_size=4,
                                    workers=4)

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model_history = model.fit(X_ngram, y_ngram, callbacks=[print_callback], epochs=50, batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5000 samples
Epoch 1/50
3712/5000 [=====================>........] - ETA: 5s - loss: 8.9393 - acc: 0.0504

In [ ]:
keras_model.fit_model(model, words, INPUT_SEQUENCE_LEN, OUTPUT_SEQUENCE_LEN, SentenceGenerator)

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(12,12))
ax[0] = sns.lineplot(x=range(10),y=model_history.history['acc'])
ax[1] = sns.lineplot(x=range(10),y=model_history.history['loss'])
#plt.legend(labels=['acc','loss'])

In [ ]:
start_index = random.randint(0, len(text_seq) - INPUT_SEQUENCE_LEN - 1)
sentence = ' '.join(text_seq[start_index: start_index + INPUT_SEQUENCE_LEN])
print(sentence)
x_pred = np.zeros((1, INPUT_SEQUENCE_LEN, len(words)))
for i in range(0,50):
    x_pred = np.zeros((1, INPUT_SEQUENCE_LEN, len(words)))
    for t, word in enumerate(sentence.split(' ')):
        x_pred[0, t, word_indices[word]] = 1.
    #print('Here the sentence is {}'.format(sentence))

    preds = model.predict(x_pred)
    next_words = indices_word[sample(preds)]
    sentence = ' '.join(sentence.split(' ')[1:])+ ' ' +next_words
    #print('After appending next word the sentence is: {}'.format(sentence))
    sys.stdout.write(' ' + next_words)
    sys.stdout.flush()
    print()

In [ ]:
indices_word[np.argmax(model.predict(X_ngram[775].reshape(1,5,10080)))]

In [ ]:
for i in range(0,500, STEP):
    x_pred = np.zeros((1, INPUT_SEQUENCE_LEN, len(words)))
    for t, word in enumerate(sentence.split(' ')):
        x_pred[0, t, word_indices[word]] = 1.
    #print('Here the sentence is {}'.format(sentence))

    preds = model.predict(x_pred)
    next_indices = sample(preds, temperature=18.0)
    try:
        next_words = ' '.join([indices_word[ind] for ind in next_indices])
    except:
        next_words = indices_word[next_indices]
    sentence = ' '.join(sentence.split(' ')[STEP:])+ ' ' +next_words
    #print('After appending next word the sentence is: {}'.format(sentence))
    sys.stdout.write(' ' + next_words)
    sys.stdout.flush()

In [ ]:
[indices_word[np.argmax(a)] for a in X_ngram[57,:,:]], indices_word[np.argmax(y_ngram[57,:])]

In [ ]:
np.argmax(y_ngram[0,:])

In [ ]:
y_pred.shape

In [ ]:
sample_seq_seq(y_pred, temperature=1.0)

In [ ]:
for i in range(0,50, STEP):
    x_pred = np.zeros((1, INPUT_SEQUENCE_LEN, len(words)))
    sentence = ' '.join(sentence)
    for t, word in enumerate(sentence.split(' ')):
        x_pred[0, t, word_indices[word]] = 1.

    preds = model.predict(x_pred)
    next_indices = sample_seq_seq(preds)
    next_words = ' '.join([indices_word[ind] for ind in next_indices])

    sentence = ' '.join(sentence.split(' ')[1:]) + ' ' +next_words
    
    sys.stdout.write(' ' + next_words)
    sys.stdout.flush()

In [ ]:
np.argmax(model.predict(x_pred), axis=2)

In [ ]:
np.random.multinomial(1, model.predict(x_pred)[1:1:],1)

In [ ]:
' '.join(['Blah']) + ' ' + '3rd_blah'